This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '9QXUgyEX2syhcZ5_patS'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
payload_test = {'limit': 1, 'api_key': API_KEY}
r_test = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=payload_test)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_test.json()['dataset_data']

{'limit': 1,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2000-06-07',
 'end_date': '2019-06-07',
 'frequency': 'daily',
 'data': [['2019-06-07',
   None,
   87.2,
   85.45,
   86.95,
   None,
   138663.0,
   12010778.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
# 1)
payload = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=payload)
print("Collected data from Frankfurt Stock Exchange for ticker: AFX_X for the year 2017")
# 2)
dataset_data = r.json()['dataset_data']
print("Converted json object to python dictionary")
# 3)
opening_prices = [data_per_day[1] for data_per_day in dataset_data['data'] if data_per_day[1] is not None]
highest_opening_price = max(opening_prices)
lowest_opening_price = min(opening_prices)
print("highest opening price: {}".format(highest_opening_price))
print("lowest opening price: {}".format(lowest_opening_price))
# 4) 
daily_changes = [data_per_day[2] - data_per_day[3] for data_per_day in dataset_data['data']
                 if (data_per_day[2] is not None) and (data_per_day[3] is not None)]
largest_daily_change = max(daily_changes)
print("largest daily change: {}".format(largest_daily_change))
# 5)
changes = []
for i in range(len(dataset_data['data']) - 1):
    change = dataset_data['data'][i + 1][4] - dataset_data['data'][i][4]
    changes.append(change)
largest_change = max(changes)
print("Largest change between any two days (based on closing price): {}".format(largest_change))
# 6)
daily_trading_vols = [data_per_day[6] for data_per_day in dataset_data['data']]
mean_daily_trading_vol = (sum(daily_trading_vols)) // (len(daily_trading_vols))
print("mean daily trading volume: {}".format(mean_daily_trading_vol))
# 7)
def max_heapify(A, heap_length, i):
    left_leaf = 2 * i + 1
    right_leaf = 2 * i + 2
    largest = i
    if (left_leaf < heap_length) and (A[left_leaf] > A[largest]):
        largest = left_leaf
    if (right_leaf < heap_length) and (A[right_leaf] > A[largest]):
        largest = right_leaf
    if largest != i:
        A[i], A[largest] = A[largest], A[i]
        max_heapify(A, heap_length, largest)

def build_heap(A):
    heap_length = len(A)
    for i in range (heap_length,-1,-1):
        max_heapify(A, heap_length, i)

def heap_sort(A):
    heap_length = len(A)
    build_heap(A)
    for i in range(heap_length - 1,0,-1):
        A[0], A[i] = A[i], A[0]
        heap_length = heap_length - 1
        max_heapify(A, heap_length, 0)

def median(data):
    heap_sort(data)
    if len(data) % 2 == 0:
        median = (data[(len(data) / 2)] + data[((len(data) / 2) - 1)]) / 2
        return median
    else:
        median = data[(len(data) // 2)]
        return median

daily_trading_vol_median = median(daily_trading_vols)
print("median daily trading volume: {}".format(daily_trading_vol_median))


Collected data from Frankfurt Stock Exchange for ticker: AFX_X for the year 2017
Converted json object to python dictionary
highest opening price: 53.11
lowest opening price: 34.0
largest daily change: 2.8100000000000023
Largest change between any two days (based on closing price): 2.559999999999995
mean daily trading volume: 89124.0
median daily trading volume: 76286.0
